In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
# import jax.numpy as jnp
# import jax
# import jax.random as rnd

from pytorch.models.og.model import fermiNet
from pytorch.sampling import MetropolisHasting
from pytorch.vmc import get_energy_and_center, compute_local_energy, compute_potential_energy, batched_cdist_l2
from pytorch.pretraining_v2 import Pretrainer

from pytorch.systems import Molecule
import math

import torch as tc
# tc.set_default_dtype(tc.float64)

from tqdm import tqdm

/home/xmax/anaconda3/envs/xmax/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [3]:
# https://stackoverflow.com/questions/42212810/tqdm-in-jupyter-notebook-prints-new-progress-bars-repeatedly
n_iterations = 10000
n_samples = 1024
n_electrons = 4
n_up = 2
n_atoms = 1
device = 'cuda'
r_atoms = tc.zeros((1, 1, 3)).to(device)
z_atoms = tc.ones((1,)).to(device) * n_electrons
walkers = tc.normal(0., 1., (n_samples, n_electrons, 3)).to(device)
print('device: ', walkers.device, r_atoms.device)
mol = Molecule(r_atoms, z_atoms, n_electrons, n_up, device=device)
print(mol.atom, mol.r_atoms.device)

device:  cuda:0 cuda:0
System: 
 Device  = cuda 
 dtype   = torch.float32 
 n_atoms = 1 
 n_up    = 2 
 n_down  = 2 

converged SCF energy = -14.351880476202
[[4, (0.0, 0.0, 0.0)]] cuda:0


In [4]:
wf = fermiNet(mol)
logdet = tc.max(walkers.sum(1), dim=1, keepdims=True)[0]

Model: 
 device   = cuda 
 n_sh     = 64 
 n_ph     = 16 
 n_layers = 2 
 n_det    = 8 



In [5]:
x1 = walkers
x2 = r_atoms
x1_sq = (x1 ** 2).sum(-1, keepdims=True)
x2_sq = (x2 ** 2).sum(-1, keepdims=True)
cdist = (x1_sq.transpose(-1, -2) + x2_sq - (2 * x1.unsqueeze(1) * x2.unsqueeze(2)).sum(-1)).sqrt()

print(cdist.shape, (x1_sq.transpose(-1, -2) + x2_sq).shape, (2 * x1.unsqueeze(2) * x2.unsqueeze(1)).shape)

torch.Size([1024, 1, 4]) torch.Size([1024, 1, 4]) torch.Size([1024, 4, 1, 3])


In [6]:
print(r_atoms.device)
e_locs = compute_local_energy(wf, walkers, r_atoms,  z_atoms)


cuda:0


In [7]:
# https://stackoverflow.com/questions/42212810/tqdm-in-jupyter-notebook-prints-new-progress-bars-repeatedly
from pytorch.pretraining_v2 import Pretrainer
from pytorch.models.og.model import fermiNet
from pytorch.systems import Molecule

tc.cuda.memory_summary()
n_iterations = 10000
n_samples = 1024
n_electrons = 4
n_up = 2
n_atoms = 1
r_atoms = tc.zeros((1, 1, 3))
z_atoms = tc.ones((1,)) * n_electrons


mol = Molecule(r_atoms, z_atoms, n_electrons, n_up, device='cuda')
walkers = mol.initialise_walkers(n_walkers=1024)

wf = fermiNet(mol, n_det=1, n_sh=64, n_ph=16, diagonal=False)

pretrainer = Pretrainer(mol)
pretrainer.pretrain(wf, walkers, n_it=10000)




System: 
 Device  = cuda 
 dtype   = torch.float32 
 n_atoms = 1 
 n_up    = 2 
 n_down  = 2 

converged SCF energy = -14.351880476202
Model: 
 device   = cuda 
 n_sh     = 64 
 n_ph     = 16 
 n_layers = 2 
 n_det    = 1 

initialized pretraining sampler
initialized sampler
tensor(-6.3479, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.4557, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.5678, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.4556, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.5406, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.6294, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.7044, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.7864, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.6874, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.8344, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.8104, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.8546, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-6.7699, device='cuda

tensor(-7.3521, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.6014, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4878, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4639, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4240, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4185, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4686, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4958, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4345, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.6880, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4584, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.8496, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.6250, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.5486, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.7212, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.5907, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4689, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4687

tensor(-7.3780, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.3152, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4027, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4680, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.3751, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4159, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4381, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.5722, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4591, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.8185, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.6579, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.6350, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.3734, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.3839, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.2249, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.5135, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4091, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4383

tensor(-7.4876, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4945, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4734, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.5714, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.3795, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4784, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.3818, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.2049, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.3640, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.6223, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.5570, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4520, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.3633, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.4665, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.3764, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.1945, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.3363, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-7.8250

pretraining:   0%|          | 0/10000 [00:00<?, ?it/s]

step 0 | e_mean -7.5310  | loss 0.32 
step 1 | e_mean -7.2638  | loss 0.33 
step 2 | e_mean -7.3577  | loss 0.35 
step 3 | e_mean -7.4852  | loss 0.35 
step 4 | e_mean -7.6206  | loss 0.36 
step 5 | e_mean -7.3778  | loss 0.36 
step 6 | e_mean -7.7514  | loss 0.34 
step 7 | e_mean -7.4669  | loss 0.35 
step 8 | e_mean -7.6512  | loss 0.35 
step 9 | e_mean -7.6565  | loss 0.34 
step 10 | e_mean -7.4818  | loss 0.35 
step 11 | e_mean -7.6096  | loss 0.39 
step 12 | e_mean -7.7876  | loss 0.39 
step 13 | e_mean -7.6861  | loss 0.41 
step 14 | e_mean -7.4740  | loss 0.42 
step 15 | e_mean -7.3849  | loss 0.41 
step 16 | e_mean -7.2693  | loss 0.42 
step 17 | e_mean -7.4650  | loss 0.43 
step 18 | e_mean -7.5322  | loss 0.43 
step 19 | e_mean -7.6332  | loss 0.42 
step 20 | e_mean -7.5368  | loss 0.46 
step 21 | e_mean -7.3021  | loss 0.45 
step 22 | e_mean -7.3789  | loss 0.45 
step 23 | e_mean -7.6273  | loss 0.46 
step 24 | e_mean -7.2323  | loss 0.46 
step 25 | e_mean -7.2319  | loss 0.

step 209 | e_mean -7.6028  | loss 0.48 
step 210 | e_mean -7.5088  | loss 0.48 
step 211 | e_mean -7.7122  | loss 0.48 
step 212 | e_mean -7.8593  | loss 0.49 
step 213 | e_mean -7.6914  | loss 0.48 
step 214 | e_mean -7.7024  | loss 0.47 
step 215 | e_mean -7.6920  | loss 0.48 
step 216 | e_mean -7.7132  | loss 0.49 
step 217 | e_mean -7.8542  | loss 0.49 
step 218 | e_mean -7.9383  | loss 0.50 
step 219 | e_mean -8.0562  | loss 0.49 
step 220 | e_mean -7.8327  | loss 0.48 
step 221 | e_mean -7.7288  | loss 0.48 
step 222 | e_mean -7.6135  | loss 0.46 
step 223 | e_mean -7.5969  | loss 0.47 
step 224 | e_mean -7.7389  | loss 0.48 
step 225 | e_mean -7.8154  | loss 0.49 
step 226 | e_mean -7.7435  | loss 0.48 
step 227 | e_mean -7.5461  | loss 0.51 
step 228 | e_mean -7.7596  | loss 0.50 
step 229 | e_mean -7.5927  | loss 0.51 
step 230 | e_mean -8.0344  | loss 0.54 
step 231 | e_mean -7.9186  | loss 0.53 
step 232 | e_mean -7.9092  | loss 0.56 
step 233 | e_mean -7.8702  | loss 0.57 


step 414 | e_mean -9.1619  | loss 0.44 
step 415 | e_mean -9.1742  | loss 0.42 
step 416 | e_mean -9.1901  | loss 0.41 
step 417 | e_mean -9.1486  | loss 0.42 
step 418 | e_mean -9.3015  | loss 0.40 
step 419 | e_mean -9.3536  | loss 0.40 
step 420 | e_mean -9.6634  | loss 0.41 
step 421 | e_mean -9.3768  | loss 0.40 
step 422 | e_mean -9.4758  | loss 0.40 
step 423 | e_mean -9.5651  | loss 0.39 
step 424 | e_mean -9.2615  | loss 0.38 
step 425 | e_mean -9.3409  | loss 0.36 
step 426 | e_mean -9.2692  | loss 0.37 
step 427 | e_mean -9.4488  | loss 0.36 
step 428 | e_mean -9.3096  | loss 0.37 
step 429 | e_mean -9.4384  | loss 0.37 
step 430 | e_mean -9.5389  | loss 0.36 
step 431 | e_mean -9.4862  | loss 0.35 
step 432 | e_mean -9.6446  | loss 0.38 
step 433 | e_mean -9.4137  | loss 0.37 
step 434 | e_mean -9.4967  | loss 0.38 
step 435 | e_mean -9.5188  | loss 0.40 
step 436 | e_mean -9.4954  | loss 0.39 
step 437 | e_mean -9.5102  | loss 0.39 
step 438 | e_mean -9.5708  | loss 0.39 


step 616 | e_mean -11.4651  | loss 0.26 
step 617 | e_mean -11.3626  | loss 0.26 
step 618 | e_mean -11.4749  | loss 0.23 
step 619 | e_mean -11.6331  | loss 0.23 
step 620 | e_mean -11.2859  | loss 0.23 
step 621 | e_mean -11.4232  | loss 0.25 
step 622 | e_mean -11.1486  | loss 0.25 
step 623 | e_mean -11.4117  | loss 0.25 
step 624 | e_mean -11.0962  | loss 0.25 


KeyboardInterrupt: 

In [ ]:
import torch as tc
from pytorch.models.og.model import fermiNet
from pytorch.pretraining_v2 import Pretrainer
from pytorch.systems import Molecule

n_el = 4
r_atoms = tc.zeros((1, 1, 3))
z_atoms = tc.ones((1,)) * n_el

mol = Molecule(r_atoms, z_atoms, n_el, device='cuda')
wf = fermiNet(mol, n_det=4, n_sh=64, n_ph=16, diagonal=False)
pretrainer = Pretrainer(mol)
pretrainer.pretrain(wf, n_it=100000)



In [8]:
1 + 2 * 3

7

In [ ]:
import random

wf = fermiNet(mol, n_det=1)
logdet = tc.max(walkers.sum(1), dim=1, keepdims=True)[0]

def l1(x, y):
    diff = (x - y).abs().sum()
    print(diff)
    return diff

def confirm_antisymmetric(wf, n_walkers=100):
    up_idxs = range(0, wf.n_up)
    down_idxs = range(wf.n_up, wf.n_el)
    
    walkers = tc.normal(0., 1., (5, wf.n_el, 3))
    l1(walkers, walkers)
    logpsi = wf(walkers)
    print(logpsi)
    
    # swap two spin up
    idxs = np.array(range(0, wf.n_el))
    (i, j) = random.sample(up_idxs, 2)
    idxs[i], idxs[j] = j, i
    w_up = walkers[:, idxs, :]
    o_up, o_down = wf.generate_orbitals(walkers)
    print(o_up[0], '\n', o_down[0])
    o_up, o_down = wf.generate_orbitals(w_up)
    print(o_up[0], '\n', o_down[0])
    l1(w_up, walkers)
    logpsi_up = wf(w_up)
    print(idxs)
    print(tc.abs(logpsi + logpsi_up).sum())
    print(logpsi_up)
    
    idxs = np.array(range(0, wf.n_el))
    (i, j) = random.sample(down_idxs, 2)
    idxs[i], idxs[j] = j, i
    w_down = walkers[:, idxs, :]
    logpsi_down = wf(w_down)
    print(idxs)
    print(tc.abs(logpsi + logpsi_down).sum())
    print(logpsi_down)
    
    
# confirm_antisymmetric(wf)